# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'



## Keygeneration

In [2]:
poly_modulus = 1024
bit_sizes= [40]
plain_modulus = 1024
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
print(ctx.param.coeff_modulus)

[1099511592961]


In [4]:
# print(len(sk.data))
# print('secret key values : ', sk.data)

In [5]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [6]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,1000)
ri2 = random.randint(0,1000)
ri3 = random.randint(0,1000)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 1]     [0, 0, 1, 0, 0, 1, 1]     [0, 0, 1, 0, 0, 1, 0, 1, 1, 1]


### Decodes back to Integer

In [7]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

528
100
932


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [8]:
encrypter = Encryptor(ctx, pk)

In [9]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [10]:
decrypter = Decryptor(ctx, sk)

In [11]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [12]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

528     100     932


## Evaluator

In [13]:
eval = Evaluator(ctx)

In [14]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

628


In [15]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

628


In [16]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

628


### Verify result

In [17]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [18]:
result = eval._mul_cipher_cipher(ct1, ct2)
print('multiplication result: ', result.data)
print('\n\n')

result = decrypter.decrypt(result)
print('decrypter result: ', result.data)
print('\n\n')

result = int_encoder.decode(result)
print('final result: ', result)
print('\n\n')

result:  [[[14, 144, 344, 539, 426, 821, 997, 610, 799, 685, 174, 975, 352, 264, 952, 123, 492, 287, 950, 635, 856, 939, 424, 448, 73, 952, 487, 967, 18, 114, 478, 1011, 406, 903, 722, 911, 936, 951, 417, 785, 876, 505, 59, 212, 294, 461, 385, 210, 31, 792, 45, 633, 882, 1023, 726, 873, 655, 368, 945, 564, 728, 21, 439, 990, 203, 427, 378, 478, 792, 241, 651, 61, 885, 666, 451, 351, 187, 411, 945, 266, 101, 866, 81, 1019, 268, 392, 980, 246, 164, 240, 275, 880, 270, 30, 922, 942, 472, 830, 305, 234, 309, 3, 95, 164, 669, 168, 286, 303, 650, 1020, 310, 77, 122, 310, 952, 830, 413, 263, 322, 487, 343, 591, 439, 19, 264, 703, 756, 405, 999, 254, 501, 967, 168, 293, 213, 515, 518, 380, 690, 93, 939, 64, 206, 1000, 601, 725, 905, 844, 121, 113, 304, 94, 533, 382, 708, 182, 823, 803, 252, 840, 722, 629, 317, 552, 125, 490, 256, 294, 189, 169, 594, 320, 809, 170, 76, 523, 14, 869, 375, 899, 940, 351, 200, 187, 756, 939, 333, 132, 308, 182, 147, 604, 935, 117, 377, 713, 695, 989, 114, 117, 6, 